In [2]:
import os
import sys

import pandas as pd
import numpy as np

import networkx as nx

from basicmemnet import memnet
from basicmemnet import plot_graph

from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.pipeline import plot_losses

In [3]:
md_train = memnet.DSL()
md_test = memnet.DSL()

md_train.import_gml("train_graph.gml")
md_test.import_gml("test_graph.gml")

test = md_test.get_graph()
train = md_train.get_graph()

### Id - link - id

In [66]:
### Prepare data - put it into appropriate format for Pykeen
def get_triplets(G):
    triples = []
    for u, v, data in G.edges(data=True):
        relation = data.get('link_type')
        triples.append([u, relation, v])
    return np.array(triples)

train_triplets = get_triplets(train)
test_triplets = get_triplets(test)
print('Number of train triplets: ', len(train_triplets))
print('Number of test triplets: ',len(test_triplets))
print(test_triplets[:3])

train_triples_factory = TriplesFactory.from_labeled_triples(train_triplets)
test_triples_factory = TriplesFactory.from_labeled_triples(test_triplets)

Number of train triplets:  25897
Number of test triplets:  6680
[['668f64fb8cc61c379809bfe8' 'has_actor' '668f64fb8cc61c379809bfe9']
 ['668f64fb8cc61c379809bfe8' 'has_element' '668f64fb8cc61c379809bfea']
 ['668f64fb8cc61c379809bfe8' 'has_element' '668f64fb8cc61c379809bfeb']]


In [27]:
test_triples_factory.relation_to_id['has_element']

1

In [67]:
from pykeen.models import ConvE

model = ConvE( train_triples_factory,
    embedding_dim       = 50, #200
    input_channels      = 1,
    output_channels     = 32,
    embedding_height    = 5, #10
    embedding_width     = 10, #20
    kernel_height       = 3,
    kernel_width        = 3,
    input_dropout       = 0.2,
    feature_map_dropout = 0.2,
    output_dropout      = 0.3,
)

from torch.optim import Adam
optimizer = Adam(params=model.get_grad_params())
from pykeen.training import LCWATrainingLoop
training_loop = LCWATrainingLoop(model=model, triples_factory=train_triples_factory, optimizer=optimizer)
# Train
losses = training_loop.train(triples_factory=train_triples_factory, num_epochs=150, batch_size=256)
# Evaluate the model
from pykeen.evaluation import RankBasedEvaluator
evaluator = RankBasedEvaluator()
metric_result = evaluator.evaluate(
    model=model,
    mapped_triples=test_triples_factory.mapped_triples,
    additional_filter_triples=train_triples_factory.mapped_triples,
    batch_size=8192,
    pre_filtered_triples=False,
    restrict_relations_to = [test_triples_factory.relation_to_id['has_element']]
)


The ConvE model should be trained with inverse triples.
This can be done by defining the TriplesFactory class with the _create_inverse_triples_ parameter set to true.
No random seed is specified. This may lead to non-reproducible results.


Training epochs on cpu:   0%|          | 0/150 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/67 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/67 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/67 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/67 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/67 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/67 [00:00<?, ?batch/s]

Evaluating on cpu:   0%|          | 0.00/2.34k [00:00<?, ?triple/s]

Encountered tensors on device_types={'cpu'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).


In [69]:
import json
with open('doctests/parentAction_ConvE_id_dim200_epoch150.json', 'w') as fp:
    json.dump(metric_result.to_dict(), fp)

In [70]:
metric_result.to_dict()

{'head': {'optimistic': {'arithmetic_mean_rank': 4714.384023921401,
   'inverse_arithmetic_mean_rank': 0.00021211678873122536,
   'inverse_geometric_mean_rank': 0.00034919943181129393,
   'z_arithmetic_mean_rank': 12.543072175712098,
   'inverse_harmonic_mean_rank': 0.005337808428405603,
   'adjusted_arithmetic_mean_rank_index': 0.14968619820399653,
   'z_geometric_mean_rank': 14.475990106623442,
   'adjusted_arithmetic_mean_rank': 0.8503408009290869,
   'variance': 10354584.013567919,
   'geometric_mean_rank': 2863.693090257937,
   'adjusted_inverse_harmonic_mean_rank': 0.004449687141008559,
   'harmonic_mean_rank': 187.3428043386523,
   'inverse_median_rank': 0.00022578460149017836,
   'count': 2341.0,
   'median_absolute_deviation': 4094.9473275124724,
   'adjusted_geometric_mean_rank_index': 0.2984765179472295,
   'median_rank': 4429.0,
   'standard_deviation': 3217.8539453443063,
   'z_inverse_harmonic_mean_rank': 17.7075961447867,
   'hits_at_1': 0.0021358393848782574,
   'hits_a

### Word - link - word

In [59]:
def get_triplets(G):
    triples = []
    for u, v, data in G.edges(data=True):
        relation = data.get('link_type')
        head = G.nodes[u].get('utterances')[0]
        tail = G.nodes[v].get('utterances')[0]
        triples.append([head, relation, tail])
    return np.array(triples)

train_triplets = get_triplets(train)
test_triplets = get_triplets(test)
print('Number of train triplets: ', len(train_triplets))
print('Number of test triplets: ',len(test_triplets))
print(test_triplets[:3])

train_triples_factory = TriplesFactory.from_labeled_triples(train_triplets)
test_triples_factory = TriplesFactory.from_labeled_triples(test_triplets)

from pykeen.models import ConvE

model = ConvE( train_triples_factory,
    embedding_dim       = 200, #200  50
    input_channels      = 1,
    output_channels     = 32,
    embedding_height    = 10,  #10    5
    embedding_width     = 20, #20   10
    kernel_height       = 3,
    kernel_width        = 3,
    input_dropout       = 0.2,
    feature_map_dropout = 0.2,
    output_dropout      = 0.3,
)

from torch.optim import Adam
optimizer = Adam(params=model.get_grad_params())
from pykeen.training import LCWATrainingLoop
training_loop = LCWATrainingLoop(model=model, triples_factory=train_triples_factory, optimizer=optimizer)
# Train
losses = training_loop.train(triples_factory=train_triples_factory, num_epochs=200, drop_last = False,)
# Evaluate the model
from pykeen.evaluation import RankBasedEvaluator
evaluator = RankBasedEvaluator()
metric_result = evaluator.evaluate(
    model=model,
    mapped_triples=test_triples_factory.mapped_triples,
    additional_filter_triples=train_triples_factory.mapped_triples,
    pre_filtered_triples=False,
    restrict_relations_to = [test_triples_factory.relation_to_id['has_element']]
)


The ConvE model should be trained with inverse triples.
This can be done by defining the TriplesFactory class with the _create_inverse_triples_ parameter set to true.
No random seed is specified. This may lead to non-reproducible results.


Number of train triplets:  25897
Number of test triplets:  6680
[['task_1_k_cooking' 'has_actor' 'subject_1']
 ['task_1_k_cooking' 'has_element' 'idle']
 ['task_1_k_cooking' 'has_element' 'approach']]


Training epochs on cpu:   0%|          | 0/200 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/1 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/1 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/1 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/1 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/1 [00:00<?, ?batch/s]

Evaluating on cpu:   0%|          | 0.00/67.0 [00:00<?, ?triple/s]

Encountered tensors on device_types={'cpu'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).


In [64]:
metric_result.to_dict()

{'head': {'optimistic': {'arithmetic_mean_rank': 7.597014925373134,
   'inverse_arithmetic_mean_rank': 0.13163064833005894,
   'inverse_geometric_mean_rank': 0.18661969011970328,
   'z_arithmetic_mean_rank': 8.290088499059529,
   'inverse_harmonic_mean_rank': 0.3097177545331512,
   'adjusted_arithmetic_mean_rank_index': 0.6039426523297492,
   'z_geometric_mean_rank': 5.902819606188657,
   'adjusted_arithmetic_mean_rank': 0.430262045646661,
   'variance': 29.46446870126977,
   'geometric_mean_rank': 5.358491375473676,
   'adjusted_inverse_harmonic_mean_rank': 0.21494383756013874,
   'harmonic_mean_rank': 3.2287461256695997,
   'inverse_median_rank': 0.14285714285714285,
   'count': 67.0,
   'median_absolute_deviation': 4.447806655516806,
   'adjusted_geometric_mean_rank_index': 0.6565353339833757,
   'median_rank': 7.0,
   'standard_deviation': 5.428118338915408,
   'z_inverse_harmonic_mean_rank': 8.54959724340807,
   'hits_at_1': 0.19402985074626866,
   'hits_at_3': 0.2537313432835821,

In [62]:
import json
with open('doctests/parentAction_ConvE_word_dim200_epoch200.json', 'w') as fp:
    json.dump(metric_result.to_dict(), fp)

### id - link - id, id - has_utterance - word

In [32]:
### Prepare data - put it into appropriate format for Pykeen
def get_triplets(G):
    triples = []
    for u, v, data in G.edges(data=True):
        relation = data.get('link_type')
        triples.append([u, relation, v])
        
        head = G.nodes[u].get('utterances')[0]
        tail = G.nodes[v].get('utterances')[0]
        
        triples.append([u, 'has_utterance', head])
        triples.append([v, 'has_utterance', tail])
    return np.array(triples)

train_triplets = get_triplets(train)
test_triplets = get_triplets(test)
print('Number of train triplets: ', len(train_triplets))
print('Number of test triplets: ',len(test_triplets))
print(test_triplets[:3])

train_triples_factory = TriplesFactory.from_labeled_triples(train_triplets)
test_triples_factory = TriplesFactory.from_labeled_triples(test_triplets)

# Model will be trained on the whole test set, but evaluation will be made only for 'has_element' link, 
# because we want to predictions for the parent actions.
results = pipeline(
    training=train_triples_factory,
    testing=test_triples_factory,
    model='TransE',
    epochs=250,  
)
results.save_to_directory('doctests/parentAction_transe_id_word_250epochs')

Number of train triplets:  77691
Number of test triplets:  20040
[['668f64fb8cc61c379809bfe8' 'has_actor' '668f64fb8cc61c379809bfe9']
 ['668f64fb8cc61c379809bfe8' 'has_utterance' 'task_1_k_cooking']
 ['668f64fb8cc61c379809bfe9' 'has_utterance' 'subject_1']]


INFO:pykeen.pipeline.api:Using device: None


Training epochs on cpu:   0%|          | 0/250 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/145 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/145 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/145 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/145 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/145 [00:00<?, ?batch/s]

Evaluating on cpu:   0%|          | 0.00/9.52k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 575.60s seconds
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=11129, num_relations=5, create_inverse_triples=False, num_triples=36985) to file:///C:/Users/maria/Downloads/MA_Thesis/Knowledge_Refinement_1/doctests/parentAction_transe_id_word_250epochs/training_triples
INFO:pykeen.pipeline.api:Saved to directory: C:\Users\maria\Downloads\MA_Thesis\Knowledge_Refinement_1\doctests\parentAction_transe_id_word_250epochs


In [33]:
results.metric_results.to_dict()

{'head': {'optimistic': {'inverse_median_rank': 0.0005223295899712719,
   'median_absolute_deviation': 2832.5115384549526,
   'adjusted_geometric_mean_rank_index': 0.870349052042363,
   'inverse_geometric_mean_rank': 0.0018956930089192776,
   'geometric_mean_rank': 527.5115724407791,
   'adjusted_arithmetic_mean_rank': 0.5883553264546012,
   'z_geometric_mean_rank': 85.10403322667139,
   'z_inverse_harmonic_mean_rank': 579.3991241453347,
   'adjusted_arithmetic_mean_rank_index': 0.4117192747746421,
   'inverse_arithmetic_mean_rank': 0.0003079676470928857,
   'inverse_harmonic_mean_rank': 0.07320301679190093,
   'standard_deviation': 3459.5515043528903,
   'arithmetic_mean_rank': 3247.0943277310926,
   'adjusted_inverse_harmonic_mean_rank': 0.07237191770342559,
   'harmonic_mean_rank': 13.66063918981326,
   'variance': 11968496.611270346,
   'count': 9520.0,
   'z_arithmetic_mean_rank': 69.56449931876959,
   'median_rank': 1914.5,
   'hits_at_1': 0.006092436974789916,
   'hits_at_3': 0.

In [12]:
# R E S U L T S
#                   Hits@1  Hits@3  Hits@5
# id-link-id:       
# word-link-word:   
# id-link-id,
# id-link-word:    